In [10]:
##All the imports
import numba
from numba import cuda

#from ipynb.fs.full.readRaw import readVolume
import json
import os
import numpy as np
import math
import matplotlib.pyplot as plt
import matplotlib.colors as colors


from sklearn.cluster import AffinityPropagation 
from sklearn import metrics 

import timeit
from PIL import Image

In [11]:
print(cuda.gpus)
cuda.select_device(1)

<Managed Device 0>, <Managed Device 1>, <Managed Device 2>, <Managed Device 3>


<weakproxy at 0x7f3bf212e7c0 to Device at 0x7f3c204125e0>

In [12]:
def ParseJSON(fname):
    contents = dict()
    # Opening JSON file -- used this as reference: https://www.geeksforgeeks.org/convert-json-to-dictionary-in-python/
    with open(fname) as json_file: 
        contents = json.load(json_file) 

        # Print the type of data variable 
        print("Type:", type(contents)) 


    #Store the information
    volumeInfo = {}
    volumeInfo['name']    = contents["name"]
    volumeInfo['type']    = contents["type"]
    volumeInfo['size']    = contents["size"]
    volumeInfo['spacing'] = contents["spacing"]
    volumeInfo['fname']   = os.path.basename(contents["url"])
    
    
    ##This uses code from here: https://stackoverflow.com/questions/1124810/how-can-i-find-path-to-given-file
    for root, dirs, files in os.walk(r'../../'):
        for name in files:
            if name == volumeInfo['fname']:
                print(os.path.abspath(os.path.join(root, name)))
                volumeInfo['dpath']   = os.path.abspath(os.path.join(root, name))


    print(volumeInfo)
    return volumeInfo

##number of bytes for a datatype
def SizeOf(dataType):

    if (dataType == "uchar"):
        return 1
    if (dataType == "char"):
        return 1
    if (dataType == "uint8"):
        return 1
    if (dataType == "uint16"):
        return 2
    if (dataType == "uint32"):
        return 4
    if (dataType == "uint64"):
        return 8
    if (dataType == "int8"):
        return 1
    print("Error: Unrecognized type", dataType)



def ParseRaw(volumeInfo):
    print(volumeInfo['type'])
    voxel_size = SizeOf(volumeInfo['type'])
    nbytes_per_line = volumeInfo['size'][0] * voxel_size
    num_of_lines    = volumeInfo['size'][1] * volumeInfo['size'][2]
    fname = volumeInfo['dpath']
    
    ##this is hardcoded to int values
    data_ptr = np.zeros(volumeInfo['size'], dtype = np.int)

    
    print("Reading the RAW file")
    
    file = open(fname, "rb") ##it's a binary file
    
    for line in range(num_of_lines):
        #print("\n\n", [char for char in file.read(nbytes_per_line)])
        y_idx = line % nbytes_per_line
        z_idx = line // volumeInfo['size'][1]
        #print(line, y_idx, z_idx)
        data_ptr[:][y_idx][z_idx] = [char for char in file.read(nbytes_per_line)]
        
    file.close()
    return data_ptr

def readVolume(fname):
    volumeInfo = ParseJSON(fname)
    data_ptr   = ParseRaw(volumeInfo)
    return data_ptr, volumeInfo


def iDivUp(a, b):
    val = (a / b + 1) if  (a % b != 0) else (a / b);
    return int(val)


#Time to do gradient calculation
@cuda.jit
def setGradients(data_ptr, grad_ptr, grad_mag_ptr, data_size):
    i = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    j = cuda.blockIdx.y*cuda.blockDim.y + cuda.threadIdx.y
    k = cuda.blockIdx.z*cuda.blockDim.z + cuda.threadIdx.z

    x_size = data_size[0]
    y_size = data_size[1]
    z_size = data_size[2]

    if ((i >= x_size) or (j >= y_size) or (k >= z_size)):
        return
    
    if (i == 0):
        x = ((data_ptr[i+1][j][k]) - (data_ptr[i][j][k]) )  / 1
    elif (i == (x_size -1)):
        x = ((data_ptr[i][j][k])   - (data_ptr[i-1][j][k])) / 1
    else:
        x = (data_ptr[i+1][j][k]   - data_ptr[i-1][j][k])   / 2
    
    if (j == 0):
        y = ((data_ptr[i][j+1][k]) - (data_ptr[i][j][k]) )  / 1
    elif (j == (x_size -1)):
        y = ((data_ptr[i][j][k])   - (data_ptr[i][j-1][k])) / 1
    else:
        y = (data_ptr[i][j+1][k]   - data_ptr[i][j-1][k])   / 2
    
    if (k == 0):
        z = ((data_ptr[i][j][k+1]) - (data_ptr[i][j][k]) )  / 1
    elif (k == (x_size -1)):
        z = ((data_ptr[i][j][k])   - (data_ptr[i][j][k-1])) / 1
    else:
        z = (data_ptr[i][j][k+1]   - data_ptr[i][j][k-1])   / 2    

    grad_ptr[i][j][k][0] = x;
    grad_ptr[i][j][k][1] = y;
    grad_ptr[i][j][k][2] = z;
    grad_mag_ptr[i][j][k] = math.sqrt(pow(x, 2) + pow(y, 2) + pow(z, 2))
    
    
    
def setGradientsCPU(data_ptr, data_size):
    grad_ptr = np.zeros([data_size[0], data_size[1], data_size[2], 3],  dtype = np.float32)
    grad_mag_ptr = np.zeros([data_size[0], data_size[1], data_size[2]], dtype = np.float32)
   
    #Figure out launch configuration
    blockSize = 8, 8, 8
    gridSize  = iDivUp(data_size[0], 8), iDivUp(data_size[1], 8), iDivUp(data_size[2], 8)
    
    setGradients[gridSize, blockSize](data_ptr, grad_ptr, grad_mag_ptr, data_size)

    cuda.synchronize()
    
    return grad_ptr, grad_mag_ptr
    
    

In [13]:
##Load in the data
fname = "../data/bonsai.json"
data_ptr, volumeInfo = readVolume(fname)
data_size = np.asarray(volumeInfo['size'])
grad_ptr, grad_mag_ptr = setGradientsCPU(data_ptr, data_size)


Type: <class 'dict'>
/home/toodemuy/projects/dvrTFGen/DVRTransferFunctionGenerator/data/bonsai_256x256x256_uint8.raw
{'name': 'Bonsai', 'type': 'uint8', 'size': [256, 256, 256], 'spacing': [1, 1, 1], 'fname': 'bonsai_256x256x256_uint8.raw', 'dpath': '/home/toodemuy/projects/dvrTFGen/DVRTransferFunctionGenerator/data/bonsai_256x256x256_uint8.raw'}
uint8
Reading the RAW file


In [15]:
##This is a CUDA kernel
@cuda.jit
def getSample(fval_arr, gval_arr, x_steps, y_steps, x_size, y_size, isTest):   
    pixelX = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    pixelY = cuda.blockIdx.y*cuda.blockDim.y + cuda.threadIdx.y
    
    ##Make sure we're  not out of bounds
    if ((pixelX >= x_size) or (pixelY >= y_size)):
        return
    x_coord = pixelX * x_steps;
    y_coord = pixelY * y_steps;
    
    
    fval = 0
    
    if (isTest):
        fval = evaluateFuncAt(x_coord, y_coord)
        gvalx, gvaly = evaluateTestGradAt(x_coord, y_coord)
    else:
        pass #not yet implemented but this would be the sibson's interpolant
    
    ##Let's set the function value!
    fval_arr[pixelX][pixelY] = fval
    gval_arr[pixelX][pixelY][0] = gvalx
    gval_arr[pixelX][pixelY][1] = gvaly

    


##This is a CUDA kernel
@cuda.jit
def setRGBA(rgba_arr, fval_arr, gval_arr, x_size, y_size, fmin, frange,
            gminx, grangex, gminy, grangey):
    pixelX = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    pixelY = cuda.blockIdx.y*cuda.blockDim.y + cuda.threadIdx.y
    
    ##Make sure we're  not out of bounds
    if ((pixelX >= x_size) or (pixelY >= y_size)):
        return
    
    ##color and opacity
    fval = fval_arr[pixelX][pixelY]
    #fval = (1- ((fval - fmin)/frange))
    fval = (fval - fmin)/frange
    
    gvalx = gval_arr[pixelX][pixelY][0]
    gvalx = (gvalx - gminx)/grangex
    
    gvaly = gval_arr[pixelX][pixelY][1]
    gvaly = (gvaly - gminy)/grangey
    
    #Lighting model
    dotP = gvalx*0.5 + gvaly*0.5
    outColour = 255 * fval * dotP

    rgba_arr[pixelX][pixelY][0] = outColour
    rgba_arr[pixelX][pixelY][1] = outColour
    rgba_arr[pixelX][pixelY][2] = outColour
    rgba_arr[pixelX][pixelY][3] = 255 # * ((fval - fmin)/frange)

##Set coordinates for the mesh
##Returns the coordinates that the grid mesh will be evaluated on
def setGridMesh(pheight, pwidth, x_lim, y_lim):
    x_steps = x_lim/pwidth
    y_steps = y_lim/pheight
    print("xsteps is %f, ysteps is %f" % (x_steps, y_steps))
    return (x_steps, y_steps)

def rayCastPixel(pixelArray, height, width, x_steps, y_steps, max_steps,
                 d_size_u, d_size_v, data_size):
    pixelX = cuda.blockIdx.x*cuda.blockDim.x + cuda.threadIdx.x
    pixelY = cuda.blockIdx.y*cuda.blockDim.y + cuda.threadIdx.y
    
    ##Make sure we're  not out of bounds
    if ((pixelX >= x_size) or (pixelY >= y_size)):
        return   
    
    
    x_size = data_size[0]
    y_size = data_size[1]
    z_size = data_size[2]
    
    levoyK = 0.95
    
    float u = d_size_u/width * pixelX
    float v = d_size_u/width * pixelY
    
    #Direction we're raycasting through
    float varPos = 0
    
    for i in range(max_steps):
        sampleVoxel()
        temp_RGBA = rgbaTF(f_val, g_val)
        
        cur_RGBA = temp_RGBA
        
        #alpha times sampled colour
        cur_RGBA[0] = cur_RGBA[3]
        cur_RGBA[1] = cur_RGBA[3]
        cur_RGBA[2] = cur_RGBA[3]
        
        #update RGBA
        finalPixel = finalPixel + (cur_RGBA)*(1 - finalPixel[3])
        
        if (finalPixel[3] > OP_THRESHOLD):
            break
            
        #Let's step to the next spot, depth-wise    
        varPos = varPos + step
        
    pixelArray[pixelX][pixelY][0] = finalPixel[0]
    pixelArray[pixelX][pixelY][1] = finalPixel[1]
    pixelArray[pixelX][pixelY][2] = finalPixel[2]
    pixelArray[pixelX][pixelY][3] = finalPixel[3]          
    
    


def populatePixelArray(pixelArray, height, width, x_steps, y_steps):
    ##For each pixel array, determine the fval at that location
    ##getFval()

    #Figure out launch configuration
    blockSize = 16, 16
    gridSize  = iDivUp(width, 16), iDivUp(height, 16)
    
    fval_arr = np.zeros([height, width], dtype=np.float32)
    gval_arr = np.zeros([height, width, 2], dtype=np.float32)


    getSample[gridSize, blockSize](fval_arr, gval_arr, x_steps, y_steps, width, height)
    #print("f_val is now", fval_arr[0][511])
    
    ##Given the fval, calculate the colour and opacity -- generate a basic gray scale image
    fval_min = fval_arr.min()
    fval_max = fval_arr.max()
    fval_range = fval_max - fval_min
        
    gval_minx = gval_arr[:, :, 0].min()
    gval_maxx = gval_arr[:, :, 0].max()
    gval_rangex = gval_maxx - gval_minx
    
    gval_miny = gval_arr[:, :, 1].min()
    gval_maxy = gval_arr[:, :, 1].max()
    gval_rangey = gval_maxy - gval_miny
    
    setRGBA[gridSize, blockSize](pixelArray, fval_arr, gval_arr, width, height, 
                                 fval_min, fval_range, gval_minx, gval_rangex,
                                 gval_miny, gval_rangey)
    cuda.synchronize();

def displayImage(pixelArray):
    img = Image.fromarray(pixelArray, 'RGBA')
    imgplot = plt.imshow(img)
    plt.show()

In [16]:
height = 512
width = 512
x_steps, y_steps = setGridMesh(height, width, 1, 1)
pixelArray = np.zeros([height, width, 4], dtype=np.uint8)
populatePixelArray(pixelArray, height, width, x_steps, y_steps)

xsteps is 0.001953, ysteps is 0.001953


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Untyped global name 'evaluateFuncAt': cannot determine Numba type of <class 'numba.core.ir.UndefinedType'>

File "<ipython-input-15-ab95d1a5f288>", line 17:
def getSample(fval_arr, gval_arr, x_steps, y_steps, x_size, y_size, isTest):   
    <source elided>
    if (isTest):
        fval = evaluateFuncAt(x_coord, y_coord)
        ^
